In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import rc, rcParams

%matplotlib inline

rc('text', usetex=True)

rcParams['font.family'] = 'Times New Roman'
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
model_prop_val_map = {
    '0040000': 'r',
    '0050000': 'g',
    '0060000': 'b',
}
model_prop = 'c'

emb_loss_prop_val_map = {
    'none': None,
    'contrastive': None,
    'triplet': None,
}

solver_prop_val_map = {
    'original': '$',
    'feature_emb': None,
}

def rand_between(size, a, b):
    return (np.random.rand(size) * (b - a) + a).tolist()

n_vals = 12

models = ('0040000', '0050000', '0060000')
emb_losses = ('none', 'contrastive', 'triplet')
solvers = ('original', 'feature_emb')

data = {
    'mota': rand_between(n_vals, 68, 77),
    'motp': rand_between(n_vals, 13, 22),
    'model': [np.random.choice(models) for _ in range(n_vals)],
    'loss': [np.random.choice(emb_losses) for _ in range(n_vals)],
    'solver': [np.random.choice(solvers) for _ in range(n_vals)],
}

df = pd.DataFrame.from_dict(data)
df

In [ ]:
def plot_tracker_loss_solver_comparison(
    df, x_col, y_col, x_label, y_label, x_units='\%', y_units='\%'
):
    def _build_axis_label(text, units=None):
        label = rf'$\textbf{{{text}}}$'
        if units is not None:
            label += f' [{units}]'
        return label

    fig, ax = plt.subplots(figsize=(12, 8), nrows=1, ncols=1)

    base_colors =  list(mcolors.BASE_COLORS.keys())
    unique_models = df['model'].unique().tolist()

    size = 25 ** 2
    for solver, group_solver_df in df.groupby(['solver']):
        assert solver in ('original', 'feature_emb')

        if solver == 'original':
            facecolors = 'none'
            solver_text = 'orig'
        else:
            facecolors = edgecolors = None
            solver_text = 'fNMS'
        
        for loss, group_loss_df in group_solver_df.groupby(['loss']):
            assert loss in ('none', 'contrastive', 'triplet')
        
            if loss == 'none':
                marker = 'o'
            elif loss == 'contrastive':
                marker = 'X'
            else:  # triplet
                marker = '^'
        
            for model, group_model_df in group_loss_df.groupby(['model']):
                xs, ys = group_model_df[x_col], group_model_df[y_col]    
                
                model_text = 'final' if 'final' in model else str(int(model))
                curr_color = base_colors[unique_models.index(model)]
                label = rf'${solver_text}, {loss}, {model_text}$'
                if facecolors == 'none':
                    color = None
                    edgecolors = curr_color
                else:
                    color = curr_color
                
                ax.scatter(
                    xs, ys, s=size, c=color, label=label, marker=marker,
                    facecolors=facecolors, edgecolors=edgecolors, linewidth=3,
                    alpha=0.75
                )
    
    x_label_formatted = _build_axis_label(x_label, x_units)
    y_label_formatted = _build_axis_label(y_label, y_units)
    
    ax.set_title(r'$\textbf{Tracker Performance Comparison}$')
    ax.set_xlabel(x_label_formatted)
    ax.set_ylabel(y_label_formatted)
    ax.legend(loc='lower left', prop={'size': 19})

    return fig

fig = plot_tracker_loss_solver_comparison(df, 'mota', 'motp', 'MOTA', 'MOTP')
fig.show()